In [5]:
import os

from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer

from dotenv import load_dotenv

In [6]:
load_dotenv(".env")
groq_api_key = os.getenv("API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# **Setup Neo4j**

In [8]:
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

# **Load LLM from Groq**

In [9]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

# **Insert Data to GraphDB**

In [10]:
text="""
Soekarno (born June 6, 1901 – June 21, 1970) was the first President of Indonesia, serving from 1945 to 1967.
He played a pivotal role in the country's struggle for independence from the Netherlands and is considered one
of the most important figures in Indonesian history. Soekarno was a charismatic orator and leader, known for
his vision of a united Indonesia based on the principles of Pancasila, which he introduced as the ideological
foundation of the state.

Born in Surabaya, Soekarno was educated in Dutch schools and later attended the Technische Hoogeschool
(now Bandung Institute of Technology), where he studied civil engineering. During his studies, he became
active in nationalist movements and began advocating for Indonesian independence.

In 1945, Soekarno, along with Mohammad Hatta, proclaimed Indonesia's independence. As president, he
sought to unify the diverse archipelago under a single government. However, his presidency faced numerous
challenges, including political instability, economic difficulties, and regional rebellions. In 1965, a coup
attempt led to significant changes in Indonesia's political landscape, eventually resulting in Soekarno's
departure from power and the rise of Suharto.

Despite his controversial later years, Soekarno remains a revered figure in Indonesia, celebrated as the
"Proclaimer of Independence" and a symbol of national unity and resilience.
 """
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content='\nSoekarno (born June 6, 1901 – June 21, 1970) was the first President of Indonesia, serving from 1945 to 1967. \nHe played a pivotal role in the country\'s struggle for independence from the Netherlands and is considered one \nof the most important figures in Indonesian history. Soekarno was a charismatic orator and leader, known for \nhis vision of a united Indonesia based on the principles of Pancasila, which he introduced as the ideological \nfoundation of the state.\n\nBorn in Surabaya, Soekarno was educated in Dutch schools and later attended the Technische Hoogeschool \n(now Bandung Institute of Technology), where he studied civil engineering. During his studies, he became \nactive in nationalist movements and began advocating for Indonesian independence.\n\nIn 1945, Soekarno, along with Mohammad Hatta, proclaimed Indonesia\'s independence. As president, he \nsought to unify the diverse archipelago under a single government. However, his pres

In [11]:
llm_transformer=LLMGraphTransformer(llm=llm)

In [12]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [13]:
# graph_documents
graph_documents[0].nodes


[Node(id='Soekarno', type='Person', properties={}),
 Node(id='Indonesia', type='Country', properties={}),
 Node(id='Surabaya', type='City', properties={}),
 Node(id='Netherlands', type='Country', properties={}),
 Node(id='Pancasila', type='Ideology', properties={}),
 Node(id='Mohammad Hatta', type='Person', properties={}),
 Node(id='Bandung Institute Of Technology', type='University', properties={}),
 Node(id='Suharto', type='Person', properties={})]

In [16]:
graph_documents[0].relationships

[Relationship(source=Node(id='Soekarno', type='Person', properties={}), target=Node(id='Indonesia', type='Country', properties={}), type='PRESIDENT_OF', properties={}),
 Relationship(source=Node(id='Soekarno', type='Person', properties={}), target=Node(id='Netherlands', type='Country', properties={}), type='LED_STRUGGLE_FOR_INDEPENDENCE_FROM', properties={}),
 Relationship(source=Node(id='Soekarno', type='Person', properties={}), target=Node(id='Pancasila', type='Ideology', properties={}), type='INTRODUCED', properties={}),
 Relationship(source=Node(id='Soekarno', type='Person', properties={}), target=Node(id='Mohammad Hatta', type='Person', properties={}), type='PROCLAIMED_INDEPENDENCE_WITH', properties={}),
 Relationship(source=Node(id='Soekarno', type='Person', properties={}), target=Node(id='Surabaya', type='City', properties={}), type='BORN_IN', properties={}),
 Relationship(source=Node(id='Soekarno', type='Person', properties={}), target=Node(id='Bandung Institute Of Technology',

In [17]:
graph.add_graph_documents(graph_documents)

In [34]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {id: STRING}
Country {id: STRING}
City {id: STRING}
Ideology {id: STRING}
University {id: STRING}
Relationship properties:

The relationships:
(:Person)-[:PRESIDENT_OF]->(:Country)
(:Person)-[:LED_STRUGGLE_FOR_INDEPENDENCE_FROM]->(:Country)
(:Person)-[:INTRODUCED]->(:Ideology)
(:Person)-[:PROCLAIMED_INDEPENDENCE_WITH]->(:Person)
(:Person)-[:BORN_IN]->(:City)
(:Person)-[:EDUCATED_AT]->(:University)
(:Person)-[:PRECEDED_BY_IN_POWER]->(:Person)
(:Country)-[:HAD_LEADERSHIP_OF]->(:Person)


# **GraphRAG - Langchain**

In [35]:
chain=GraphCypherQAChain.from_llm(llm=llm,
                                 graph=graph,
                                 verbose=True,
                                 top_k=10,
                                 allow_dangerous_requests=True # Add this line to acknowledge the risk
                                )

In [37]:
response=chain.invoke({"query":"Soekarno is President of ?"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (soekarno:Person {id: 'Soekarno'})-[:PRESIDENT_OF]->(country:Country) RETURN country
Full Context:
[{'country': {'id': 'Indonesia'}}]

> Finished chain.


{'query': 'Soekarno is President of ?',
 'result': 'Soekarno is President of Indonesia.'}

In [41]:
response=chain.invoke({"query":"What Ideology Soekarno introduced ?"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Soekarno"})-[:INTRODUCED]->(i:Ideology) RETURN i
Full Context:
[{'i': {'id': 'Pancasila'}}]

> Finished chain.


{'query': 'What Ideology Soekarno introduced ?', 'result': 'Pancasila.'}

In [42]:
response=chain.invoke({"query":"Where was Soekarno born?"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Soekarno"})-[:BORN_IN]->(c:City) RETURN c
Full Context:
[{'c': {'id': 'Surabaya'}}]

> Finished chain.


{'query': 'Where was Soekarno born?',
 'result': 'Soekarno was born in Surabaya.'}